In [1]:
import sys
import torch

sys.path.insert(0, "..")
basedir = "../.."

from common.config import create_object, load_config

%matplotlib widget

torch._dynamo.config.suppress_errors = True
torch._dynamo.disable()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# dconfig = load_config("../autoencoder/configs/data/burgersshift.yaml")
# dconfig.datasize.spacedim = 1
# dset = create_object(dconfig)

In [6]:
dconfig = load_config("../autoencoder/configs/data/droplettrigpoly1.yaml")
dconfig.datasize.spacedim = 1
dset = create_object(dconfig)

#dset.downsample_time(10)
#dset.downsample(4)

In [48]:
import time
import glob
import datetime
import copy
import os
import pickle
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import seaborn as sns
import matplotlib.cm as cm

import itertools

from itertools import combinations
from sklearn.decomposition import PCA
from sklearn.kernel_ridge import KernelRidge
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from copy import deepcopy

import utils

class ETINetHelper():
  def __init__(self, config):
    self.update_config(config)

  def update_config(self, config):
    self.config = deepcopy(config)

  def create_etinet(self, dataset, k, config=None, **args):
    if config is None:
      config = self.config

    assert(len(dataset.data.shape) < 4)
    if len(dataset.data.shape) == 3:
      din = dataset.params.shape[-1]
      dout = dataset.data.shape[-1]

    td = args.get("td", None)
    seed = args.get("seed", 0)
    device = args.get("device", 0)

    recclass = globals()[args.get("recclass", config.recclass)]
    recparams = copy.deepcopy(dict(args.get("recparams", config.recparams)))

    aeclass = globals()[args.get("aeclass", config.aeclass)]
    aeparams = copy.deepcopy(dict(args.get("aeparams", config.aeparams)))

    recparams["seq"][0] = k + 1
    recparams["seq"][-1] = dout

    return ETINet(dataset, k, aeclass, aeparams, recclass, recparams, td=td, seed=seed, device=device)

  @staticmethod
  def get_operrs(etinet, times=None, testonly=False):
    if testonly:
      data = etinet.dataset.data[etinet.numtrain:,]
    else:
      data = etinet.dataset.data

    inputs = torch.tensor(data[:, 0]).to(etinet.device)

    encode = etinet.aenet.encode(inputs)
    errors = etinet.get_errors(encode, data[:, 1:], times=times, aggregate=False)

    return errors
  
  @staticmethod
  def plot_op_predicts(etinet, testonly=False, xs=None, cmap="viridis", topdown=True):
    if testonly:
      data = etinet.dataset.data[etinet.numtrain:,]
    else:
      data = etinet.dataset.data

    if xs == None:
      xs = np.linspace(0, 1, len(data[0, 0]))

    inputs = etinet.aenet.encode(torch.tensor(data[:, 0]).to(etinet.device))
    predicts = etinet.propagate(inputs).cpu().detach().numpy()

    errors = []
    n = predicts.shape[0]
    for s in range(data[:, 1:].shape[1]):
      currpredict = predicts[:, s].reshape((n, -1))
      currreference = data[:, s].reshape((n, -1))
      errors.append(np.mean(np.linalg.norm(currpredict - currreference, axis=1) / np.linalg.norm(currreference, axis=1)))
        
    print(f"Average Relative L2 Error over all times: {np.mean(errors):.4f}")

    if len(data.shape) == 3:
      if topdown:
        fig, axes = plt.subplots(1, 3, figsize=(9, 3))
         
      else:
        fig, ax = plt.subplots(figsize=(4, 3))

    if topdown:
      @widgets.interact(i=(0, n-1))
      def plot_interact(i=0):
        if len(data.shape) == 3:
          axes[0].clear()
          axes[1].clear()
          axes[2].clear()

          fig.suptitle(f"RelL2 {np.linalg.norm(predicts[i, s-1] - data[i, s]) / np.linalg.norm(data[i, s])}")

          axes[0].set_title("Exact")
          axes[0].imshow(data[i, 1:], cmap=cmap, origin="lower", aspect='auto')

          axes[1].set_title("Predict")
          axes[1].imshow(predicts[i, 1:], cmap=cmap, origin="lower", aspect='auto')

          axes[2].set_title("|Exact - Predict|")
          axes[2].imshow(np.abs(data[i, 1:] - predicts[i]), cmap=cmap, origin="lower", aspect='auto')
       
    else:
      @widgets.interact(i=(0, n-1), s=(1, etinet.T-1))
      def plot_interact(i=0, s=1):
        print(f"Avg Relative L2 Error for t0 to t{s}: {errors[s-1]:.4f}")

        if len(data.shape) == 3:
          ax.clear()
          ax.set_title(f"RelL2 {np.linalg.norm(predicts[i, s-1] - data[i, s]) / np.linalg.norm(data[i, s])}")
          ax.plot(xs, data[i, 0], label="Input", linewidth=1)
          ax.plot(xs, predicts[i, s-1], label="Predicted", linewidth=1)
          ax.plot(xs, data[i, s], label="Exact", linewidth=1)
          ax.legend()

  @staticmethod
  def plot_errorparams(etinet, param=-1):
    if param == -1:
        # Auto-detect one varying parameter
        param = 0
        P = etinet.dataset.params.shape[1]
        for p in range(P):
            if np.abs(etinet.dataset.params[0, p] - etinet.dataset.params[1, p]) > 0:
                param = p
                break

    l2error = np.asarray(ETINetHelper.get_operrs(etinet, times=[etinet.T - 1]))
    params = etinet.dataset.params

    print(params.shape, l2error.shape)

    if isinstance(param, (list, tuple, np.ndarray)) and len(param) == 2:
        # 3D scatter plot for 2 varying parameters
        x = params[:, param[0]]
        y = params[:, param[1]]
        z = l2error

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        sc = ax.scatter(x, y, z, c=z, cmap='viridis', s=10)

        ax.set_xlabel(f"Param {param[0]}")
        ax.set_ylabel(f"Param {param[1]}")
        ax.set_zlabel("Operator Error")
        fig.colorbar(sc, ax=ax, label="Operator Error")

    else:
      # Fallback to 2D scatter if param is 1D
      fig, ax = plt.subplots()
      ax.scatter(params[:, param], l2error, s=2)
      ax.set_xlabel(f"Parameter {param}")
      ax.set_ylabel("Operator Error")

    fig.tight_layout()

class ETINet():
  def __init__(self, dataset, k, aeclass, aeparams, recclass, recparams, td, seed, device):
    self.dataset = dataset
    self.device = device
    self.td = td
    self.k = k
  
    if self.td is None:
      self.prefix = f"{self.dataset.name}{str(recclass.__name__)}ETINet"
    else:
      self.prefix = self.td

    torch.manual_seed(seed)
    np.random.seed(seed)
    self.seed = seed

    datacopy = self.dataset.data.copy()
    self.numtrain = int(datacopy.shape[0] * 0.8)
    
    self.T = self.dataset.data.shape[1]
    self.trainarr = datacopy[:self.numtrain]
    self.testarr = datacopy[self.numtrain:]
    self.optparams = None

    self.datadim = len(self.dataset.data.shape) - 2
    self.aestep = 0
    self.recstep = 0

    aeparams["encodeSeq"][0] = self.dataset.data.shape[-1]
    aeparams["encodeSeq"][-1] = self.k
    aeparams["decodeSeq"][0] = self.k
    aeparams["decodeSeq"][-1] = self.dataset.data.shape[-1]

    recparams["seq"][0] = self.k + 1
    recparams["seq"][-1] = self.dataset.data.shape[-1]

    self.aeclass = aeclass
    self.aeparams = copy.deepcopy(aeparams)
    self.recclass = recclass
    self.recparams = copy.deepcopy(recparams)

    self.aenet = aeclass(**aeparams).float().to(device)
    self.recnet = recclass(**recparams).float().to(device)

    self.metadata = {
      "aeclass": aeclass.__name__,
      "aeparams": aeparams,
      "recclass": recclass.__name__,
      "recparams": recparams,
      "dataset_name": dataset.name,
      "data_shape": list(dataset.data.shape),
      "data_checksum": float(np.sum(dataset.data)),
      "seed": seed,
      "epochs": []
    }

  def reconstruct(self, z, ts):
    z_shape = z.shape
    *leading_dims, N = z_shape
    T = ts.shape[0]

    z_expanded = z.unsqueeze(-2).expand(*leading_dims, T, N)

    t_shape = [1] * len(leading_dims) + [T, 1]
    t_expanded = ts.view(*t_shape).expand(*leading_dims, T, 1)

    result = torch.cat([z_expanded, t_expanded], dim=-1)

    recon = self.recnet(result)
    return recon

  def propagate(self, code, start=1, end=-1):
    fullts = torch.linspace(0, 1, self.T).float().to(self.device)
    
    if end > 0:
      ts = fullts[start:end+1]
    else:
      ts = fullts[start:]

    out = self.reconstruct(code, ts)
    return out

  def get_errors(self, testarr, testrest, ords=(2,), times=None, aggregate=True):
    assert(aggregate or len(ords) == 1)
    
    if isinstance(testarr, np.ndarray):
      testarr = torch.tensor(testarr, dtype=torch.float32)

    if isinstance(testrest, np.ndarray):
      testrest = torch.tensor(testrest, dtype=torch.float32)

    if times is None:
      times = range(self.T-1)
  
    out = self.propagate(testarr)

    n = testarr.shape[0]
    orig = testrest.cpu().detach().numpy()
    out = out.cpu().detach().numpy()

    print(orig.shape, out.shape)

    if aggregate:
      orig = orig.reshape([n, -1])
      out = out.reshape([n, -1])
      testerrs = []
      for o in ords:
        testerrs.append(np.mean(np.linalg.norm(orig - out, axis=1, ord=o) / np.linalg.norm(orig, axis=1, ord=o)))

      return tuple(testerrs)
    
    else:
      o = ords[0]
      testerrs = []

      if len(times) == 1:
        t = times[0]
        origslice = orig[:, t-1].reshape([n, -1])
        outslice = out[:, t-1].reshape([n, -1])
        return np.linalg.norm(origslice - outslice, axis=1, ord=o) / np.linalg.norm(origslice, axis=1, ord=o)
      else:
        for t in range(orig.shape[1]):
          origslice = orig[:, t].reshape([n, -1])
          outslice = out[:, t].reshape([n, -1])
          testerrs.append(np.mean(np.linalg.norm(origslice - outslice, axis=1, ord=o) / np.linalg.norm(origslice, axis=1, ord=o)))

        return testerrs
      
  def get_ae_errors(self, testarr, ords=(2,)):
    if isinstance(testarr, np.ndarray):
      testarr = torch.tensor(testarr, dtype=torch.float32)
  
    out = self.aenet(testarr).cpu().detach().numpy()
    orig = testarr.cpu().detach().numpy()

    testerrs = []
    for o in ords:
      testerrs.append(np.mean(np.linalg.norm(orig - out, axis=1, ord=o) / np.linalg.norm(orig, axis=1, ord=o)))

    return tuple(testerrs)

  def load_models(self, filename_prefix, verbose=False, min_epochs=0):
    search_path = f"savedmodels/etinet/{filename_prefix}*.pickle"
    matching_files = glob.glob(search_path)

    print("Searching for model files matching prefix:", filename_prefix)
    if not hasattr(self, "metadata"):
        raise ValueError("Missing self.metadata. Cannot match models without metadata. Ensure model has been initialized with same config.")

    for addr in matching_files:
      try:
          with open(addr, "rb") as handle:
              dic = pickle.load(handle)
      except Exception as e:
          if verbose:
              print(f"Skipping {addr} due to read error: {e}")
          continue

      meta = dic.get("metadata", {})
      is_match = all(
          meta.get(k) == self.metadata.get(k)
          for k in self.metadata.keys()
      )

      # Check if model meets the minimum epoch requirement
      model_epochs = meta.get("epochs")
      if model_epochs is None:
          if verbose:
              print(f"Skipping {addr} due to missing epoch metadata.")
          continue
      elif isinstance(model_epochs, list):  # handle legacy or list format
          if sum(model_epochs) < min_epochs:
              if verbose:
                  print(f"Skipping {addr} due to insufficient epochs ({sum(model_epochs)} < {min_epochs})")
              continue
      elif model_epochs < min_epochs:
          if verbose:
              print(f"Skipping {addr} due to insufficient epochs ({model_epochs} < {min_epochs})")
          continue

      if is_match:
          print("Model match found. Loading from:", addr)
          self.recnet.load_state_dict(dic["recnet"])
          self.aenet.load_state_dict(dic["aenet"])
          self.metadata["epochs"] = meta.get("epochs")
          if "opt" in dic:     
            self.optparams = dic["opt"]

          return True
      elif verbose:
          print("Metadata mismatch in file:", addr)
          for k in self.metadata:
              print(f"{k}: saved={meta.get(k)} vs current={self.metadata.get(k)}")

    print("Load failed. No matching models found.")
    print("Searched:", matching_files)
    return False

  def train_recnet(self, epochs, save=True, optim=torch.optim.AdamW, lr=1e-4, printinterval=10, batch=32, ridge=0, loss=None, best=True, verbose=False):
    def recnet_epoch(dataloader, writer=None, optimizer=None, scheduler=None, ep=0, printinterval=10, loss=None, testarr=None, testrest=None):
      losses = []
      testerrors1 = []
      testerrors2 = []
      testerrorsinf = []

      def closure(codes, rest):
        optimizer.zero_grad()

        out = self.propagate(codes)
        target = rest
        
        res = loss(out, target)
        res.backward()
        
        if writer is not None and self.recstep % 5 == 0:
          writer.add_scalar("main/loss", res, global_step=self.recstep)

        return res

      for codes, rest in dataloader:
        self.recstep += 1
        error = optimizer.step(lambda: closure(codes, rest))
        losses.append(float(error.cpu().detach()))

      if scheduler is not None:
        scheduler.step(np.mean(losses))

      # print test
      if printinterval > 0 and (ep % printinterval == 0):
        testerr1, testerr2, testerrinf = self.get_errors(testarr, testrest, ords=(1, 2, np.inf))
        if scheduler is not None:
          print(f"{ep+1}: Train Loss {error:.3e}, LR {scheduler.get_last_lr()[-1]:.3e}, Relative ETINet Error (1, 2, inf): {testerr1:3f}, {testerr2:3f}, {testerrinf:3f}")
        else:
          print(f"{ep+1}: Train Loss {error:.3e}, Relative ETINet Error (1, 2, inf): {testerr1:3f}, {testerr2:3f}, {testerrinf:3f}")

        if writer is not None:
            writer.add_scalar("misc/relativeL1error", testerr1, global_step=ep)
            writer.add_scalar("main/relativeL2error", testerr2, global_step=ep)
            writer.add_scalar("misc/relativeLInferror", testerrinf, global_step=ep)

      return losses, testerrors1, testerrors2, testerrorsinf
  
    assert(self.aestep > 0)

    loss = nn.MSELoss() if loss is None else loss()

    losses, testerrors1, testerrors2, testerrorsinf = [], [], [], []

    initial = torch.tensor(self.trainarr[:, 0], dtype=torch.float32).to(self.device)
    rest = torch.tensor(self.trainarr[:, 1:], dtype=torch.float32).to(self.device)
    train = self.aenet.encode(initial).detach()

    testinitial = torch.tensor(self.testarr[:, 0], dtype=torch.float32).to(self.device)
    testrest = torch.tensor(self.testarr[:, 1:], dtype=torch.float32).to(self.device)
    test = self.aenet.encode(testinitial).detach()

    opt = optim(self.recnet.parameters(), lr=lr, weight_decay=ridge)
    scheduler = lr_scheduler.ReduceLROnPlateau(opt, patience=30)
    dataloader = DataLoader(torch.utils.data.TensorDataset(train, rest), shuffle=False, batch_size=batch)

    writer = None
    if self.td is not None:
      name = f"./tensorboard/{datetime.datetime.now().strftime('%d-%B-%Y')}/{self.td}/{datetime.datetime.now().strftime('%H.%M.%S')}/"
      writer = torch.utils.tensorboard.SummaryWriter(name)
      print("Tensorboard writer location is " + name)

    print("Number of NN trainable parameters", utils.num_params(self.recnet))
    print(f"Starting ETINet rec model at {time.asctime()}...")
    print("train", train.shape, "test", test.shape)
      
    bestdict = { "loss": float(np.inf), "ep": 0 }
    for ep in range(epochs):
      lossesN, testerrors1N, testerrors2N, testerrorsinfN = recnet_epoch(dataloader, optimizer=opt, scheduler=scheduler, writer=writer, ep=ep, printinterval=printinterval, loss=loss, testarr=test, testrest=testrest)
      losses += lossesN; testerrors1 += testerrors1N; testerrors2 += testerrors2N; testerrorsinf += testerrorsinfN

      if best and ep > epochs // 2:
        avgloss = np.mean(lossesN)
        if avgloss < bestdict["loss"]:
          bestdict["recnet"] = self.recnet.state_dict()
          bestdict["opt"] = opt.state_dict()
          bestdict["loss"] = avgloss
          bestdict["ep"] = ep
        elif verbose:
          print(f"Loss not improved at epoch {ep} (Ratio: {avgloss/bestdict['loss']:.2f}) from {bestdict['ep']} (Loss: {bestdict['loss']:.2e})")
      
    print(f"Finished training ETINet rec model at {time.asctime()}...")

    if best:
      self.recnet.load_state_dict(bestdict["recnet"])
      opt.load_state_dict(bestdict["opt"])

    self.aeoptparams = opt.state_dict()
    self.metadata["epochs"].append(epochs)

    if save:
      now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

      # Compute total training epochs
      total_epochs = sum(self.metadata["epochs"]) if isinstance(self.metadata["epochs"], list) else self.metadata["epochs"]

      filename = (
          f"{self.dataset.name}_"
          f"{self.aeclass.__name__}_"
          f"{self.aeparams['encodeSeq']}_"
          f"{self.recclass.__name__}_"
          f"{self.recparams['seq']}_"
          f"{self.seed}_"
          f"{total_epochs}ep_"
          f"{now}.pickle"
      )

      dire = "savedmodels/etinet"
      addr = os.path.join(dire, filename)

      if not os.path.exists(dire):
          os.makedirs(dire)

      with open(addr, "wb") as handle:
          pickle.dump({
              "aenet": self.aenet.state_dict(),
              "recnet": self.recnet.state_dict(),
              "metadata": self.metadata,
              "opt": self.optparams
          }, handle, protocol=pickle.HIGHEST_PROTOCOL)

      print("Model saved at", addr)

    return { "losses": losses, "testerrors1": testerrors1, "testerrors2": testerrors2, "testerrorsinf": testerrorsinf }

  def train_aenet(self, epochs, save=True, optim=torch.optim.AdamW, lr=1e-4, printinterval=10, batch=32, ridge=0, loss=None, best=True, verbose=False):
    def aenet_epoch(dataloader, writer=None, optimizer=None, scheduler=None, ep=0, printinterval=10, loss=None, testarr=None):
      losses = []
      testerrors1 = []
      testerrors2 = []
      testerrorsinf = []

      def closure(codes):
        optimizer.zero_grad()

        out = self.aenet(codes)
        target = codes
        
        res = loss(out, target)
        res.backward()
        
        if writer is not None and self.recstep % 5 == 0:
          writer.add_scalar("main/aeloss", res, global_step=self.recstep)

        return res

      for codes in dataloader:
        self.aestep += 1
        error = optimizer.step(lambda: closure(codes))
        losses.append(float(error.cpu().detach()))

      if scheduler is not None:
        scheduler.step(np.mean(losses))

      # print test
      if printinterval > 0 and (ep % printinterval == 0):
        testerr1, testerr2, testerrinf = self.get_ae_errors(testarr, ords=(1, 2, np.inf))
        if scheduler is not None:
          print(f"{ep+1}: Train Loss {error:.3e}, LR {scheduler.get_last_lr()[-1]:.3e}, Relative ETINet AE Error (1, 2, inf): {testerr1:3f}, {testerr2:3f}, {testerrinf:3f}")
        else:
          print(f"{ep+1}: Train Loss {error:.3e}, Relative ETINet AE Error (1, 2, inf): {testerr1:3f}, {testerr2:3f}, {testerrinf:3f}")

        if writer is not None:
            writer.add_scalar("misc/relativeL1error", testerr1, global_step=ep)
            writer.add_scalar("main/relativeL2error", testerr2, global_step=ep)
            writer.add_scalar("misc/relativeLInferror", testerrinf, global_step=ep)

      return losses, testerrors1, testerrors2, testerrorsinf
  
    loss = nn.MSELoss() if loss is None else loss()

    losses, testerrors1, testerrors2, testerrorsinf = [], [], [], []

    initial = torch.tensor(self.trainarr[:, 0], dtype=torch.float32).to(self.device)
    test = torch.tensor(self.testarr[:, 0], dtype=torch.float32).to(self.device)

    opt = optim(self.aenet.parameters(), lr=lr, weight_decay=ridge)
    scheduler = lr_scheduler.ReduceLROnPlateau(opt, patience=30)
    dataloader = DataLoader(initial, shuffle=False, batch_size=batch)

    if self.optparams is not None:
      opt.load_state_dict(self.optparams)

    writer = None
    if self.td is not None:
      name = f"./tensorboard/{datetime.datetime.now().strftime('%d-%B-%Y')}/{self.td}/{datetime.datetime.now().strftime('%H.%M.%S')}/"
      writer = torch.utils.tensorboard.SummaryWriter(name)
      print("Tensorboard writer location is " + name)

    print("Number of NN trainable parameters", utils.num_params(self.recnet))
    print(f"Starting training ETINet AE model at {time.asctime()}...")
    print("train", initial.shape, "test", test.shape)
      
    bestdict = { "loss": float(np.inf), "ep": 0 }
    for ep in range(epochs):
      lossesN, testerrors1N, testerrors2N, testerrorsinfN = aenet_epoch(dataloader, optimizer=opt, scheduler=scheduler, writer=writer, ep=ep, printinterval=printinterval, loss=loss, testarr=test)
      losses += lossesN; testerrors1 += testerrors1N; testerrors2 += testerrors2N; testerrorsinf += testerrorsinfN

      if best and ep > epochs // 2:
        avgloss = np.mean(lossesN)
        if avgloss < bestdict["loss"]:
          bestdict["aenet"] = self.aenet.state_dict()
          bestdict["opt"] = opt.state_dict()
          bestdict["loss"] = avgloss
          bestdict["ep"] = ep
        elif verbose:
          print(f"Loss not improved at epoch {ep} (Ratio: {avgloss/bestdict['loss']:.2f}) from {bestdict['ep']} (Loss: {bestdict['loss']:.2e})")
      
    print(f"Finished training ETINet AE model at {time.asctime()}...")

    if best:
      self.aenet.load_state_dict(bestdict["aenet"])
      opt.load_state_dict(bestdict["opt"])

    return { "losses": losses, "testerrors1": testerrors1, "testerrors2": testerrors2, "testerrorsinf": testerrorsinf }


In [ ]:
configs = ["etinet", "ltinet", "ldnet"]

In [50]:
import models
lconfig = load_config("../autoencoder/configs/experiments/etinet.yaml")
experiment = ETINetHelper(lconfig)

FFNet = models.FFNet
FFAutoencoder = models.FFAutoencoder
test = experiment.create_etinet(dset, 10)

test.train_aenet(500, lr=1e-3)
test.train_recnet(500, lr=1e-3)

Number of NN trainable parameters 909301
Starting training ETINet AE model at Mon Jul 21 10:41:51 2025...
train torch.Size([200, 301]) test torch.Size([50, 301])
1: Train Loss 2.187e-03, LR 1.000e-03, Relative ETINet AE Error (1, 2, inf): 0.112253, 0.136846, 0.386307
11: Train Loss 6.491e-04, LR 1.000e-03, Relative ETINet AE Error (1, 2, inf): 0.053248, 0.064079, 0.118436
21: Train Loss 6.646e-04, LR 1.000e-03, Relative ETINet AE Error (1, 2, inf): 0.052937, 0.063736, 0.113986
31: Train Loss 6.775e-04, LR 1.000e-03, Relative ETINet AE Error (1, 2, inf): 0.052982, 0.063756, 0.116930
41: Train Loss 8.687e-04, LR 1.000e-03, Relative ETINet AE Error (1, 2, inf): 0.048085, 0.058123, 0.117756
51: Train Loss 2.017e-04, LR 1.000e-03, Relative ETINet AE Error (1, 2, inf): 0.027273, 0.034397, 0.073421
61: Train Loss 1.796e-04, LR 1.000e-03, Relative ETINet AE Error (1, 2, inf): 0.026920, 0.034455, 0.071044
71: Train Loss 1.899e-04, LR 1.000e-03, Relative ETINet AE Error (1, 2, inf): 0.029123, 0.

{'losses': [0.10148518532514572,
  0.08618798106908798,
  0.06175689399242401,
  0.030077064409852028,
  0.02598160319030285,
  0.030582120642066002,
  0.01590675488114357,
  0.010357859544456005,
  0.013937491923570633,
  0.0176104549318552,
  0.018090389668941498,
  0.017687326297163963,
  0.014555457979440689,
  0.013835892081260681,
  0.014232377521693707,
  0.01554053369909525,
  0.014859921298921108,
  0.011170512065291405,
  0.011355932801961899,
  0.010686914436519146,
  0.011907569132745266,
  0.011434457264840603,
  0.011543471366167068,
  0.011633199639618397,
  0.010943735018372536,
  0.012295125983655453,
  0.010849087499082088,
  0.010548949241638184,
  0.010069601237773895,
  0.010815968737006187,
  0.011266159825026989,
  0.010436946526169777,
  0.011100124567747116,
  0.009605739265680313,
  0.009911504574120045,
  0.009968812577426434,
  0.010775329545140266,
  0.010921524837613106,
  0.009877667762339115,
  0.010793427936732769,
  0.009546171873807907,
  0.0098285097

In [11]:
ETINetHelper.plot_errorparams(test)

TypeError: ETINet.get_errors() missing 1 required positional argument: 'testrest'